In [1]:
# import packages
import h5py # read h5

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
from matplotlib import rc
from cycler import cycler

import numpy as np
import pandas as pd

from merge_h5_files import merge_preserve_time
import os
import glob

In [ ]:
import pandas as pd
str1 = 1554601887620000
str2 = 1554033665335
#trans to datetime

a = pd.to_datetime(str1, unit="us")
b = pd.to_datetime(str2, unit="ms")
print(a)
print(b)

In [2]:
f1 = h5py.File('C:/Users/fengxiang.mao/Desktop/Devon data/HFTS-1_Devon/20220707/Evo 5 (POP)/Zgabay A14H - pop - strain change.h5','r')
print(list(f1.keys()))

['data', 'depth', 'stamps', 'stamps_unix']


In [3]:

dstrain = (f1['data'])  #（time, depth）
depth = (f1['depth'])
date_str = (f1['stamps'])
date_us = (f1['stamps_unix'])


depth = np.asarray(depth)
dstrain = np.asarray(dstrain)
date_str = np.asarray(date_str)
date_us = np.asarray(date_us)

In [ ]:

# 1) 你的时间戳列（微秒）→ 带UTC时区
dt_utc = pd.to_datetime(date_us, unit='us', utc=True)  # DatetimeIndex(UTC)

# 2) 你的字符串列是 bytes，需要先解码成 str 再解析为 datetime（naive）
ds = pd.to_datetime(pd.Series(date_str.astype(str)), format='%m/%d/%Y %H:%M:%S.%f')
# print(type(ds))
# print(ds)

# 3) 告诉 pandas：这些字符串代表的是 CET（UTC+1），再转成 UTC 比较
ds_cet = ds.dt.tz_localize('Europe/Paris')  # 或 Europe/Berlin / Europe/Rome 都是 CET/CEST 区域
ds_as_utc = ds_cet.dt.tz_convert('UTC')

# 4) 比较两种来源是否对齐（应接近全 0）
diff = (ds_as_utc.to_numpy() - dt_utc.to_numpy())
print("平均差：", pd.to_timedelta(pd.Series(diff)).mean())
print("最大差：", pd.to_timedelta(pd.Series(diff)).max())
print("最小差：", pd.to_timedelta(pd.Series(diff)).min())

# 5) 如果你想统一到 America/Chicago 显示：
dt_chi     = dt_utc.tz_convert('America/Chicago')
ds_cet_chi = ds_cet.dt.tz_convert('America/Chicago')

# 现在 dt_chi 和 ds_cet_chi 就是同一时区的两套时间，应该一致
print(type(ds_cet))
print(ds_cet)
print(ds_cet.shape)
print(type(ds_cet[0]))
start = pd.Timestamp("2019-04-01 18:05:27")
print(type(start))

#判断两个变量类型是否相同
print(type(start) == type(ds_cet[0]))

平均差： 0 days 00:00:00
最大差： 0 days 00:00:00
最小差： 0 days 00:00:00
<class 'pandas.core.series.Series'>
0      2022-03-14 20:56:49.350706+01:00
1      2022-03-14 20:58:13.618471+01:00
2      2022-03-14 20:59:38.346233+01:00
3      2022-03-14 21:01:02.745833+01:00
4      2022-03-14 21:02:27.442383+01:00
                     ...               
7003   2022-03-21 18:08:26.564440+01:00
7004   2022-03-21 18:09:51.421313+01:00
7005   2022-03-21 18:11:16.399100+01:00
7006   2022-03-21 18:12:41.365719+01:00
7007   2022-03-21 18:14:06.359133+01:00
Length: 7008, dtype: datetime64[ns, Europe/Paris]
(7008,)
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
True
